# Quantum algorithm on CI graphs

## Import package

In [ ]:
import sys
import numpy as np
import math
import random
import turtle
import qiskit
from qiskit import QuantumCircuit, execute, Aer
from qiskit import IBMQ
from qiskit.aqua.algorithms import Grover
from qiskit.aqua.components.oracles import TruthTableOracle
from qiskit.providers.ibmq import least_busy

## Parameters

In [ ]:
N = 12
nearest_connect = 2 #0 < lower < upper <= N/2
farthest_connect = 4
itr = 9
prob = 0.3
plot_graph = False
state = 'local' # remote or local
token = ''

## IBMQ config

In [ ]:
if(state == 'remote'):
    IBMQ.enable_account(token)

## Set backend(local simulator or remote IBMQ)

In [ ]:
if(state != 'remote'):
    # simulator
    backend = Aer.get_backend('qasm_simulator')
else:
    provider = IBMQ.get_provider(hub='ibm-q')
    #backend = provider.get_backend('ibmq_qasm_simulator')
    backend = provider.get_backend('ibmq_bogota')

## Check parameters

In [ ]:
def check_input(N, nearest_connect, farthest_connect):
    if(nearest_connect <= 0 or farthest_connect <= 0 or N <= 0):
        print('error: nearest_connect should bigger than 0')
        exit()
    elif(nearest_connect > farthest_connect):
        print('error: nearest_connect should not bigger than farthest_connect')
        exit()
    elif(farthest_connect > N/2):
        print('error: farthest_connect should not exceed half of total node')
        exit()
    elif((prob > 1) | (prob < 0)):
        print('error: probability should be 0 to 1')
        exit()
    elif((state != 'local') & (state != 'remote')):
        print('error: state should be local or remote')
        exit()
    else:
        return

## Create Adjacency Matrix and Input Boolean array

In [ ]:
def create_matrix(N, nearest_connect, farthest_connect):
    array = np.zeros((N,N))
    for i in range (N):
        for j in range(nearest_connect, farthest_connect+1):
            if(i+j>N-1):
                array[i][i+j-N]=1
            else:
                array[i][i+j] = 1
    for i in range (N):
        for j in range(nearest_connect, farthest_connect+1):
            if(i-j<0):
                array[i][i-j+N]=1
            else:
                array[i][i-j] = 1
    return array

def decision(probability):
    return random.random() < probability

## Plotting utility: Turtle

In [ ]:
def plot(a,f):
    def draw(i,j,pos):
        turtle.penup()
        turtle.setposition(pos[j][0],pos[j][1])
        dir_vec = [pos[i][0]-pos[j][0],pos[i][1]-pos[j][1]]
        dir = math.atan(dir_vec[1]/dir_vec[0])
        #fixup
        if dir_vec[0]<0 :
            dir+=math.pi
        
        #radient to degree
        dir = dir*180/math.pi

        length = math.sqrt((dir_vec[0])*(dir_vec[0])+(dir_vec[1])*(dir_vec[1]))

        turtle.setheading(dir)
        turtle.pendown()
        turtle.forward(length)

    size = len(f)
    #remember cordinate
    table = np.zeros((size,2))
    screen = turtle.Screen()
    turtle.speed(-3)
    turtle.penup()
    turtle.setposition(100,0)
    turtle.pendown()
    turtle.setheading(90)
    turtle.hideturtle()
    degree=0
    index=0
    arc = 360/size
    #draw vertices
    while degree<360:

        if f[index]==0:
            turtle.color('blue')
        else:
            turtle.color('red')
        turtle.dot(10)
        turtle.color('black')
        turtle.write(str(index),font=("Verdana", 15, "normal"))
        table[index] = turtle.pos()
        turtle.penup()
        turtle.circle(100,extent=arc)
        turtle.pendown()
        degree+=arc
        index +=1
        
    #draw edges
    for i in range(size):
        for j in range(i):
            if a[i][j]==1:
                pass
                draw(i,j,table)

    print('Finish plotting. Please close the drawing window manually.')
    turtle.mainloop()

## Classical functions

In [ ]:
def find_min_one_idx(low_idx,high_idx,f):
    min_one_idx = -1
    for j in range(low_idx,high_idx+1):
        if(f[j]):
            min_one_idx = j
            break
    return min_one_idx

def find_max_one_idx(low_idx,high_idx,f):
    max_one_idx = -1
    for j in range(high_idx,low_idx-1,-1):
        if(f[j]):
            max_one_idx = j
            break
    return max_one_idx

def is_any_one(low_idx,high_idx,f):
    is_any_one_var = 0
    for k in range(low_idx,high_idx+1):
        if(k >= N):
            k = k - N
        if(f[k]):
            is_any_one_var = 1
            break
    return is_any_one_var

## Functions using quantum algorithm

In [ ]:
def ndarray2str(ndarray):
    string = ''
    for i in ndarray:
        string = string + str(int(i))
    return string
    
def is_any_one_quantum(low_idx,high_idx,f,iteration):
    
    doubled_f = np.concatenate((f,f))
    output_str = ndarray2str(doubled_f[low_idx:high_idx+1])
    original_output_str_len = len(output_str)
    
    if original_output_str_len == 1:
        return doubled_f[low_idx]
    
    power = math.ceil(math.log(len(output_str),2))
    output_str = output_str + '0'*(pow(2,power+1)-len(output_str))
    oracle = TruthTableOracle(output_str)
    grover = Grover(oracle)
    
    iteration_results = []
    for j in range(iteration):
        
        result = grover.run(backend, shots=1)
        counts = result['measurement']

        for i in counts:
            measurement_outcome = int(i,2)

        if measurement_outcome > original_output_str_len - 1:
            iteration_results.append(0)
        else:
            measured_idx = measurement_outcome + low_idx
            iteration_results.append(int(doubled_f[measured_idx]))
    
    num_of_one = 0
    for i in iteration_results:
        if i == 1:
            num_of_one += 1
            
    if num_of_one > iteration/2:
        return 1
    else:
        return 0
    
def find_min_one_idx_update_quantum(low_idx,high_idx,f):#if we measure 1,it returns the index of 1,else returns -1 
    #find the index of 1
    doubled_f = np.concatenate((f,f))
    output_str = ndarray2str(doubled_f[low_idx:high_idx+1])
    
    original_output_str_len = len(output_str)
    
    if original_output_str_len == 1:#low_idx = high_idx , length =1
        if (doubled_f[low_idx] == 1):
            return low_idx
        else:
            return -1
    
    power = math.ceil(math.log(len(output_str),2))
    if pow(2,power) > len(output_str):
        output_str = output_str + '0'*(pow(2,power)-len(output_str))

    oracle = TruthTableOracle(output_str)
    grover = Grover(oracle)
    
    
    result = grover.run(backend, shots=1)
    
    counts = result['measurement']
    
    for i in counts:
        measurement_outcome = int(i,2)
    if measurement_outcome > original_output_str_len - 1:
        return -1
    else:
        measured_idx = measurement_outcome + low_idx
        if(int(doubled_f[measured_idx]) == 1):
            return measured_idx
        else:
            return -1

def find_min_one_idx_quantum_sub(low_idx,high_idx,f):
    guess = -1
    current_high = high_idx
    
    #find the index of 1
    doubled_f = np.concatenate((f,f))
    output_str = ndarray2str(doubled_f[low_idx:high_idx+1])
    original_output_str_len = len(output_str)
    
    if original_output_str_len == 1: #low_idx = high_idx
        if (doubled_f[low_idx] == 1):
            return low_idx
        else:
            return -1
    
    power = math.ceil(math.log(len(output_str),2))
    if pow(2,power) > len(output_str):
        output_str = output_str + '0'*(pow(2,power)-len(output_str))
    
    oracle = TruthTableOracle(output_str)
    grover = Grover(oracle)
    
    
    result = grover.run(backend, shots=1)
    
    counts = result['measurement']
    
    for i in counts:
        measurement_outcome = int(i,2)
    if measurement_outcome > original_output_str_len - 1:
        return -1
    else:
        measured_idx = measurement_outcome + low_idx
        if (int(doubled_f[measured_idx]) == 0):#then we expect they are all 0
            return -1
        guess = measured_idx
        current_high = measured_idx
        #shrink the range
        while (current_high-low_idx > 0):

            current_high = find_min_one_idx_update_quantum(low_idx,current_high-1,f)

            if (current_high == -1): #then our guess is correct
                break
            else:
                guess = current_high

        return guess
    
def find_max_one_idx_update_quantum(original_low_idx,low_idx,high_idx,f):#if we measure 1,it returns the index of 1,else returns -1 
    #find the index of 1
    doubled_f = np.concatenate((f,f))
    output_str = ndarray2str(doubled_f[low_idx:high_idx+1])
    
    original_output_str_len = len(output_str)
    
    if original_output_str_len == 1:#low_idx = high_idx
        if (doubled_f[high_idx] == 1):
            return high_idx
        else:
            return -1
    
    power = math.ceil(math.log(len(output_str),2))
    if pow(2,power) > len(output_str):
        output_str = output_str + '0'*(pow(2,power)-len(output_str))

    oracle = TruthTableOracle(output_str)
    grover = Grover(oracle)
    

    result = grover.run(backend, shots=1)
    
    counts = result['measurement']
    
    for i in counts:
        measurement_outcome = int(i,2)
    if measurement_outcome > original_output_str_len - 1:
        return -1
    else:
        measured_idx = measurement_outcome + original_low_idx
        if(int(doubled_f[measured_idx]) == 1):
            return measured_idx
        else:
            return -1

def find_max_one_idx_quantum_sub(low_idx,high_idx,f):
    guess = -1
    current_low = low_idx
    
    #find the index of 1
    doubled_f = np.concatenate((f,f))
    output_str = ndarray2str(doubled_f[low_idx:high_idx+1])
    original_output_str_len = len(output_str)
    
    if original_output_str_len == 1: #low_idx = high_idx
        if (doubled_f[high_idx] == 1):
            return high_idx
        else:
            return -1
   
    power = math.ceil(math.log(len(output_str),2))
    if pow(2,power) > len(output_str):
        output_str = output_str + '0'*(pow(2,power)-len(output_str))
    
    oracle = TruthTableOracle(output_str)
    grover = Grover(oracle)
    
    
    result = grover.run(backend, shots=100)
    
    counts = result['measurement']

    for i in counts:
        measurement_outcome = int(i,2)
    if measurement_outcome > original_output_str_len - 1:
        return -1
    else:
        measured_idx = measurement_outcome + low_idx
        if (int(doubled_f[measured_idx]) == 0): #then we expect they are all 0
            return -1
        guess = measured_idx
        current_low = measured_idx
        #shrink the range
        while (high_idx-current_low > 0):

            current_low = find_max_one_idx_update_quantum(low_idx,current_low+1,high_idx,f)

            if (current_low == -1): #then our guess is correct
                break
            else:
                guess = current_low

        return guess

def find_min_one_idx_quantum(low_idx,high_idx,f,itr):
    exp = np.zeros(itr) #do experiment itr times
    outcome = np.zeros(itr) #it saves the outcome
    times = np.zeros(itr) #it saves the times of an outcome
    c_index = 0
    for i in range(itr):
        exp[i] = find_min_one_idx_quantum_sub(low_idx,high_idx,f)

    for i in range(itr):
        for j in range(c_index):
            if (outcome[j] == exp[i]):
                times[j] += 1
                break
        outcome[c_index] = exp[i]
        times[c_index] += 1
        c_index += 1
    
    result = -1
    current_max = -1
    for i in range(itr):
        if(times[i] > current_max):
            result = outcome[i]
            current_max = times[i]
    return int(result)

def find_max_one_idx_quantum(low_idx,high_idx,f,itr):
    exp = np.zeros(itr) #do experiment itr times
    outcome = np.zeros(itr) #it saves the outcome
    times = np.zeros(itr) #it saves the times of an outcome
    c_index = 0
    for i in range(itr):
        exp[i] = find_max_one_idx_quantum_sub(low_idx,high_idx,f)

    for i in range(itr):
        for j in range(c_index):
            if (outcome[j] == exp[i]):
                times[j] += 1
                break
        outcome[c_index] = exp[i]
        times[c_index] += 1
        c_index += 1
    
    result = -1
    current_max = -1
    for i in range(itr):
        if(times[i] > current_max):
            result = outcome[i]
            current_max = times[i]
    return int(result)
    

## main function

### Create graph and boolean input array

In [ ]:
check_input(N, nearest_connect, farthest_connect)
a = create_matrix(N, nearest_connect, farthest_connect)

f = np.zeros(N)

### Plot graph

In [ ]:
if plot_graph:
    plot(a,f)

### Divided into chunks

In [ ]:
# experiment 1: the accuracy depending on numbers of iterations in chunks
results_classical = []
results_quantum = []
for j in range(7):
    chunk_size = farthest_connect - nearest_connect
    is_collision_in_chunk_arr = np.zeros(math.ceil(N/chunk_size))
    is_collision_in_chunk_arr_quantum = np.zeros(math.ceil(N/chunk_size))
    for k in range(N):
        f[k] = decision(prob)
    #classical
    if(farthest_connect != nearest_connect):
    #     print("size::::",math.ceil(N/chunk_size))
        for i in range(math.ceil(N/chunk_size)):
            min_idx_in_chunk = chunk_size * i
            max_idx_in_chunk = min(chunk_size * (i+1) - 1,N-1)
    #         print("before min one idx:",min_idx_in_chunk,"before max one idx:",max_idx_in_chunk)
            min_one_idx_in_chunk = find_min_one_idx(min_idx_in_chunk,max_idx_in_chunk,f)
            max_one_idx_in_chunk = find_max_one_idx(min_idx_in_chunk,max_idx_in_chunk,f,)
            while ((min_one_idx_in_chunk != -1 and max_one_idx_in_chunk == -1) or (min_one_idx_in_chunk == -1 and max_one_idx_in_chunk != -1)):
                    min_one_idx_in_chunk = find_min_one_idx(min_idx_in_chunk,max_idx_in_chunk,f)
                    max_one_idx_in_chunk = find_max_one_idx(min_idx_in_chunk,max_idx_in_chunk,f)

            if(min_one_idx_in_chunk==-1):
                results_classical.append(is_collision_in_chunk_arr[i])
                continue
    #         print("min one idx:",min_idx_in_chunk,"max one idx:",max_idx_in_chunk)
            connect_low_idx = min_one_idx_in_chunk + nearest_connect
            connect_high_idx = max_one_idx_in_chunk + farthest_connect

    #         print("connect low:",connect_low_idx,"connect high",connect_high_idx)
            is_collision_in_chunk_arr[i] = is_any_one(connect_low_idx,connect_high_idx,f)
            results_classical.append(is_collision_in_chunk_arr[i])
            #is_collision_in_chunk_arr_quantum[i] = is_any_one_quantum(connect_low_idx,connect_high_idx,f)

        is_collision_var = is_any_one(0,len(is_collision_in_chunk_arr)-1,is_collision_in_chunk_arr)
       
        
    else:
        is_collision_in_chunk_arr = np.zeros(N)
        for i in range(N):
            connect_low_idx = i + nearest_connect
            connect_high_idx = i + farthest_connect
            if(f[i]):
                is_collision_in_chunk_arr[i] = is_any_one(connect_low_idx, connect_high_idx,f)
                results_classical.append(is_collision_in_chunk_arr[i])
        is_collision_var = is_any_one(0,len(is_collision_in_chunk_arr)-1,is_collision_in_chunk_arr)
        

    #quantum
    if(farthest_connect != nearest_connect):
    #     print("size::::",math.ceil(N/chunk_size))
        for i in range(math.ceil(N/chunk_size)):
            min_idx_in_chunk = chunk_size * i
            max_idx_in_chunk = min(chunk_size * (i+1) - 1,N-1)
    #         print("before min one idx:",min_idx_in_chunk,"before max one idx:",max_idx_in_chunk)
            min_one_idx_in_chunk = find_min_one_idx_quantum(min_idx_in_chunk,max_idx_in_chunk,f,itr)
            max_one_idx_in_chunk = find_max_one_idx_quantum(min_idx_in_chunk,max_idx_in_chunk,f,itr)
            while ((min_one_idx_in_chunk != -1 and max_one_idx_in_chunk == -1) or (min_one_idx_in_chunk == -1 and max_one_idx_in_chunk != -1)):
                    min_one_idx_in_chunk = find_min_one_idx_quantum(min_idx_in_chunk,max_idx_in_chunk,f,itr)
                    max_one_idx_in_chunk = find_max_one_idx_quantum(min_idx_in_chunk,max_idx_in_chunk,f,itr)

            if(min_one_idx_in_chunk==-1):
                results_quantum.append(is_collision_in_chunk_arr[i])
                continue
    #         print("min one idx:",min_idx_in_chunk,"max one idx:",max_idx_in_chunk)
            connect_low_idx = min_one_idx_in_chunk + nearest_connect
            connect_high_idx = max_one_idx_in_chunk + farthest_connect

    #         print("connect low:",connect_low_idx,"connect high",connect_high_idx)
            is_collision_in_chunk_arr[i] = is_any_one_quantum(connect_low_idx,connect_high_idx,f,itr)
            results_quantum.append(is_collision_in_chunk_arr[i])
            #is_collision_in_chunk_arr_quantum[i] = is_any_one_quantum(connect_low_idx,connect_high_idx,f)

        is_collision_var = is_any_one_quantum(0,len(is_collision_in_chunk_arr)-1,is_collision_in_chunk_arr,itr)
        
        
    else:
        is_collision_in_chunk_arr = np.zeros(N)
        for i in range(N):
            connect_low_idx = i + nearest_connect
            connect_high_idx = i + farthest_connect
            if(f[i]):
                is_collision_in_chunk_arr[i] = is_any_one_quantum(connect_low_idx, connect_high_idx,f,itr)
                results_quantum.append(is_collision_in_chunk_arr[i])
        is_collision_var = is_any_one_quantum(0,len(is_collision_in_chunk_arr)-1,is_collision_in_chunk_arr,itr)
        

        

### Print result

In [ ]:
# 原本的主程式(實驗用不到)
#print('quantum result:')
#print(is_collision_var_quantum)
# print('classical result:')
# print(is_collision_var)

#if is_collision_var == is_collision_var_quantum:
 #   print("the result of quantum algorithm is identical with classical algorithm")
#else:
 #   print("the result of quantum algorithm is different from classical algorithm")
    
# if farthest_connect != nearest_connect:
#     print("is_collision_in_chunk_arr:")
#     print(is_collision_in_chunk_arr)
#     print("is_collision_in_chunk_arr_quantum:")
#     print(is_collision_in_chunk_arr_quantum)


In [ ]:
num_of_i = 0
# print(results)
for i in range(len(results_classical)):
    if results_classical[i] == results_quantum[i]:
        num_of_i += 1
        
print("accuracy:")
print(num_of_i/len(results_classical))

In [ ]:
# experiment 2: the total accuracy depending on numbers of iterations on the condition of given 0/1 of every chunk 
classical_results = []
quantum_results = []
chunk_size = farthest_connect - nearest_connect
is_collision_in_chunk_arr = np.zeros(math.ceil(N/chunk_size))
is_collision_in_chunk_arr_quantum = np.zeros(math.ceil(N/chunk_size))
for i in range(100):
    for j in range(6):
        is_collision_in_chunk_arr[j] = decision(0.5)
    is_collision_in_chunk_arr_quantum = is_collision_in_chunk_arr

    is_collision_var = is_any_one(0,len(is_collision_in_chunk_arr)-1,is_collision_in_chunk_arr)
    is_collision_var_quantum = is_any_one_quantum(0,len(is_collision_in_chunk_arr_quantum)-1,is_collision_in_chunk_arr_quantum,1)
    classical_results.append(is_collision_var)
    quantum_results.append(is_collision_var_quantum)

In [ ]:
error_counts = 0
for i in range(100):
    if classical_results[i] != quantum_results[i]:
        error_counts += 1
print("accuracy:")
print(1-error_counts/100)

In [ ]:
# 測試bit string長短對成功率的影響
from qiskit import IBMQ
from qiskit.aqua.algorithms import Grover
from qiskit.aqua.components.oracles import TruthTableOracle
from qiskit import QuantumCircuit, execute, Aer

backend = Aer.get_backend('qasm_simulator')

expression = '0010'

oracle = TruthTableOracle(expression)

grover = Grover(oracle)
error_counts = 0
for j in range(100):
    result = grover.run(backend, shots=1)
    counts = result['measurement']
    for i in counts:
        measurement_outcome = int(i,2)
    if measurement_outcome != 2:
        error_counts += 1
print("accuracy:")
print((100-error_counts)/100)